# 2025 End of Season Rower Survey Analysis

This notebook contains the complete analysis of the Rower Survey data. It covers data loading, cleaning, processing, and visualization.

## 1. Setup and Imports
Import all the necessary libraries for the analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

## 2. Configuration
Define the file paths for the data and mapping files.

In [ ]:
file_path = "2025 End of Season Rower Survey - Responses (AR).xlsx"
mapping_file_path = "column_mapping.xlsx"
brand_colors_file = "BrandColours.md"
chart_path = "charts"

## 3. Helper Functions
This section contains all the functions used for data processing and visualization, consolidated from the original `.py` scripts.

In [ ]:
def load_data(path):
    """
    Loads data from an Excel file into a pandas DataFrame.
    """
    try:
        df = pd.read_excel(path)
        print("Data loaded successfully.")
        return df
    except FileNotFoundError:
        print(f"Error: The file '{path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

def load_column_mapping(path):
    """
    Loads column mapping from an Excel file.
    """
    try:
        mapping_df = pd.read_excel(path)
        print("Column mapping loaded successfully.")
        return mapping_df
    except FileNotFoundError:
        print(f"Error: The mapping file '{path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the mapping file: {e}")
        return None

def apply_data_types(df, mapping_df):
    """
    Applies data types to the DataFrame based on the mapping file.
    """
    type_mapping = dict(zip(mapping_df["new_name"], mapping_df["recommended_type"]))
    
    ordered_likert_columns = [
        "support_1st_place_medals_masters",
        "rating_promotion_governance",
        "rating_accessibility",
        "rating_positive_experience",
        "rating_high_performance_pathways",
    ]
    category_order = ["Strongly Disagree", "Disagree", "Neutral", "Agree", "Strongly Agree"]
    cat_dtype = pd.api.types.CategoricalDtype(categories=category_order, ordered=True)
    mapping = {i + 1: label for i, label in enumerate(category_order)}

    for col_name, dtype in type_mapping.items():
        if col_name in df.columns:
            try:
                if col_name in ordered_likert_columns:
                    df[col_name] = df[col_name].map(mapping).astype(cat_dtype)
                    print(f"Applied ordered categorical type to '{col_name}'.")
                    continue

                if pd.api.types.is_string_dtype(dtype) and dtype.startswith("Int"):
                    df[col_name] = pd.to_numeric(df[col_name], errors="coerce").astype(dtype)
                elif dtype == "float64":
                    df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
                else:
                    df[col_name] = df[col_name].astype(dtype)
            except Exception as e:
                print(f"Could not convert column '{col_name}' to '{dtype}': {e}")
    print("\nData types applied successfully.")
    return df

def get_brand_colors(file_path="BrandColours.md"):
    """
    Parses a markdown file to extract brand color hex codes for charts.
    """
    colors = []
    try:
        with open(file_path, "r") as f:
            for line in f:
                if "HEX:" in line:
                    match = re.search(r"#(?:[0-9a-fA-F]{3}){1,2}", line)
                    if match:
                        colors.append(match.group(0))
    except FileNotFoundError:
        print(f"Warning: Brand color file '{file_path}' not found. Using default colors.")
        return None
    return colors[:5] if colors else None

def create_bar_chart(df, column_name, chart_path="charts", brand_colors_file="BrandColours.md", highlight_bar=None, chart_name=None, title=None):
    """
    Creates and saves a bar chart for a given column.
    """
    if column_name not in df.columns:
        print(f"Column '{column_name}' not found in the DataFrame.")
        return

    if not os.path.exists(chart_path):
        os.makedirs(chart_path)

    brand_colors = get_brand_colors(brand_colors_file)
    default_color = brand_colors[0] if brand_colors else "#003E7E"
    highlight_color = brand_colors[2] if brand_colors and len(brand_colors) > 2 else "#FFB81C"

    if pd.api.types.is_categorical_dtype(df[column_name]) and df[column_name].cat.ordered:
        order = df[column_name].cat.categories
    else:
        order = df[column_name].value_counts().index

    palette = [highlight_color if bar == highlight_bar else default_color for bar in order] if highlight_bar and highlight_bar in order else [default_color] * len(order)

    plt.figure(figsize=(10, 6))
    sns.countplot(y=df[column_name].dropna(), order=order, palette=palette)
    plt.title(title if title else f'Distribution of Responses for "{column_name}"')
    plt.xlabel("Count")
    plt.ylabel("Response")
    plt.tight_layout()

    file_name = f"{chart_name}.png" if chart_name else f"{column_name}_distribution.png"
    save_path = os.path.join(chart_path, file_name)
    plt.savefig(save_path)
    print(f"\nChart saved to '{save_path}'")
    plt.close()

def create_comparison_chart(df, col1, col2, chart_path="charts", brand_colors_file="BrandColours.md", chart_name=None, title=None, legend_labels=("Col1", "Col2")):
    """
    Creates a grouped bar chart to compare two columns.
    """
    if col1 not in df.columns or col2 not in df.columns:
        print(f"One or both columns '{col1}', '{col2}' not found.")
        return

    melted_df = df.melt(value_vars=[col1, col2], var_name="location", value_name="preference")
    melted_df["location"].replace({col1: legend_labels[0], col2: legend_labels[1]}, inplace=True)
    
    brand_colors = get_brand_colors(brand_colors_file)
    palette = brand_colors[:2] if brand_colors and len(brand_colors) >= 2 else ["#003E7E", "#FFB81C"]
    
    preference_order = ["Strongly Prefer", "Prefer", "Neutral", "Don't Prefer", "Strongly Don't Prefer"]
    order = [p for p in preference_order if p in melted_df["preference"].unique()]

    plt.figure(figsize=(12, 7))
    sns.countplot(data=melted_df, y="preference", hue="location", order=order, palette=palette)
    plt.title(title if title else f"Comparison of {col1} and {col2}")
    plt.xlabel("Count of Responses")
    plt.ylabel("Preference")
    plt.legend(title="Location")
    plt.tight_layout()

    file_name = f"{chart_name}.png" if chart_name else f"{col1}_vs_{col2}_comparison.png"
    save_path = os.path.join(chart_path, file_name)
    if not os.path.exists(chart_path):
        os.makedirs(chart_path)
    plt.savefig(save_path)
    print(f"\nComparison chart saved to '{save_path}'")
    plt.close()

def create_reasons_summary_chart(df, reason_columns, chart_path="charts", brand_colors_file="BrandColours.md", chart_name=None, title=None):
    """
    Creates a summary bar chart for multiple boolean-like reason columns.
    """
    reason_counts = df[reason_columns.keys()].sum().sort_values(ascending=False)
    reason_counts.index = reason_counts.index.map(reason_columns)
    
    brand_colors = get_brand_colors(brand_colors_file)
    default_color = brand_colors[0] if brand_colors else "#003E7E"

    plt.figure(figsize=(10, 7))
    sns.barplot(x=reason_counts.values, y=reason_counts.index, color=default_color)
    plt.title(title if title else "Summary of Reasons")
    plt.xlabel("Number of Rowers Citing Reason")
    plt.ylabel("Reason")
    plt.tight_layout()

    file_name = f"{chart_name}.png" if chart_name else "reasons_summary.png"
    save_path = os.path.join(chart_path, file_name)
    if not os.path.exists(chart_path):
        os.makedirs(chart_path)
    plt.savefig(save_path)
    print(f"\nSummary chart saved to '{save_path}'")
    plt.close()

print("All helper functions defined.")

## 4. Data Loading and Pre-processing
Load the survey data and the column mapping file, then rename and apply the correct data types to the columns.

In [ ]:
survey_data = load_data(file_path)
mapping_df = load_column_mapping(mapping_file_path)

if survey_data is not None and mapping_df is not None:
    column_mapping = dict(zip(mapping_df["old_name"], mapping_df["new_name"]))
    survey_data.rename(columns=column_mapping, inplace=True)
    print("\nColumns renamed.")
    
    survey_data = apply_data_types(survey_data, mapping_df)
    
    print("\nData processing complete. Displaying DataFrame info:")
    survey_data.info()
    display(survey_data.head())

## 5. Filter for Masters Rowers
Create a separate DataFrame containing only the responses from Masters rowers (age 27+).

In [ ]:
if 'age_category' in survey_data.columns:
    masters_age_categories = ["27-40", "41-60", "61+"]
    masters_df = survey_data[survey_data["age_category"].isin(masters_age_categories)].copy()
    print(f"Filtered for Masters rowers. Found {len(masters_df)} responses.")
    display(masters_df.head())
else:
    print("Column 'age_category' not found. Cannot filter for Masters rowers.")

## 6. Generate Visualizations
Create and save charts for the analyzed data. Each chart is generated in its own cell.

### Masters: Season Extension

In [ ]:
create_bar_chart(
    masters_df,
    "desired_masters_season_extension",
    chart_name="desired_masters_season_extension_distribution",
    title="Desired Masters Season Extension",
)

### Masters: Location Preference (Canning Bridge vs. Champion Lakes)

In [ ]:
create_comparison_chart(
    masters_df,
    "prefer_canning_bridge_masters",
    "prefer_champion_lakes_masters",
    chart_name="location_preference_comparison_masters",
    title="Masters Rowers: Preference for Canning Bridge vs. Champion Lakes",
    legend_labels=("Canning Bridge", "Champion Lakes"),
)

### Masters: Support for 1st Place Medals

In [ ]:
create_bar_chart(
    masters_df,
    "support_1st_place_medals_masters",
    chart_name="support_1st_place_medals_masters_distribution",
    title="Support for 1st Place Medals for Masters",
)

### General Ratings (All Rowers)

In [ ]:
rating_columns = {
    "rating_promotion_governance": "Rating of Promotion and Governance",
    "rating_accessibility": "Rating of Accessibility",
    "rating_positive_experience": "Rating of Positive Experience",
    "rating_high_performance_pathways": "Rating of High-Performance Pathways",
}

for col, title in rating_columns.items():
    create_bar_chart(
        survey_data,
        col,
        chart_name=f"{col}_distribution",
        title=title,
    )

### Masters: Reasons for Not Competing

In [ ]:
reasons_columns = {
    "reason_recreational_time_commitment": "Time Commitment",
    "reason_recreational_skill_level": "Skill Level",
    "reason_recreational_cost": "Cost",
    "reason_recreational_social_aspect": "Prefer Social Aspect",
}

create_reasons_summary_chart(
    masters_df,
    reason_columns=reasons_columns,
    chart_name="masters_reasons_not_competing",
    title="Primary Reasons Masters Rowers Do Not Compete",
)

### Masters: Support for Transition to Competitive Rowing

In [ ]:
create_bar_chart(
    masters_df,
    "support_transition_to_competitive",
    chart_name="masters_support_transition_to_competitive",
    title="Support for Transitioning to Competitive Rowing (Masters)",
)